In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import copy
import argparse
import os
from PIL import Image
import torchvision
import cv2
import torchvision.transforms as transforms
import torchvision.models as models

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
num_steps = 700
style_weight = 1000000


In [3]:
class ContentLoss(nn.Module):
    def __init__(self, target, ):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [4]:
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

In [5]:
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

In [6]:
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

In [7]:
def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers,
                               style_layers):
    cnn = copy.deepcopy(cnn)

    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

In [8]:
def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer

In [9]:
def image_loader(image_name, target_size):
    image = Image.open(image_name)
    # Resize the image to the target size
    image = transforms.Resize(target_size)(image)
    # Convert to tensor and add a batch dimension
    image = transforms.ToTensor()(image).unsqueeze(0)
    return image.to(device, torch.float)

In [10]:
def get_dataset_loader(folder_path, target_size):
    # Get the list of image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    
    # Create a DataLoader for the images in the folder
    dataset_loader = []
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = image_loader(image_path, target_size)
        dataset_loader.append(image)
    
    return dataset_loader

In [11]:
def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps,
                       style_weight, content_weight=1):
    """Run the style transfer."""
    print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
                                                                     normalization_mean, normalization_std, style_img,
                                                                     content_img,content_layers_default, style_layers_default)
    optimizer = get_input_optimizer(input_img)

    print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                print()

            return style_score + content_score

        optimizer.step(closure)

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img


In [15]:
def run_style_transfer_on_dataset(cnn, normalization_mean, normalization_std,
                                  content_dataset, style_dataset, num_steps,
                                  style_weight, content_weight=1, output_path="C:/Users/AYUSH/Deep Learning Project/outputimages/"):
    for i, content_img in enumerate(content_dataset):
        for j, style_img in enumerate(style_dataset):
            # Initialize the input image as a copy of the content image
            input_img = content_img.clone()

            # Run style transfer
            output = run_style_transfer(cnn, normalization_mean, normalization_std,
                                        content_img, style_img, input_img, num_steps, style_weight, content_weight)

            # Save the output image
            torchvision.utils.save_image(output, os.path.join(output_path, f"output_{i}_{j}.jpg"))

In [16]:
def st(style_img, content_img):
    ######################################################################
    # Importing the Model
    # -------------------
    cnn = models.vgg19(pretrained=True).features.to(device).eval()

    ######################################################################
    # Additionally, VGG networks are trained on images with each channel
    # normalized by mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].
    # We will use them to normalize the image before sending it into the network.
    #

    cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
    cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)


    ######################################################################
    # Next, we select the input image. You can use a copy of the content image
    # or white noise.
    #
    input_img = content_img.clone()

    output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                       content_img, style_img, input_img, num_steps, style_weight)
    return output

In [20]:
if __name__ == "__main__":
    # Define paths to your content and style folders
    content_folder = "C:/Users/AYUSH/Deep Learning Project/content/"
    style_folder = "C:/Users/AYUSH/Deep Learning Project/style/"


    # Specify the target size for resizing images
    target_size = (250, 350)

    # Import VGG model
    cnn = models.vgg19(pretrained=True).features.to(device).eval()

    # Normalization parameters
    cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
    cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

    # Get content and style datasets
    content_dataset = get_dataset_loader(content_folder, target_size)
    style_dataset = get_dataset_loader(style_folder, target_size)

    # Run style transfer on the dataset
    run_style_transfer_on_dataset(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_dataset, style_dataset, num_steps, style_weight)

Building the style transfer model..


C:\Users\AYUSH\AppData\Local\Temp\ipykernel_13084\4145313249.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mean = torch.tensor(mean).view(-1, 1, 1)
C:\Users\AYUSH\AppData\Local\Temp\ipykernel_13084\4145313249.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.std = torch.tensor(std).view(-1, 1, 1)


Optimizing..
run [50]:
Style Loss : 19.993582 Content Loss: 7.240349

run [100]:
Style Loss : 6.942190 Content Loss: 5.707569

run [150]:
Style Loss : 4.999226 Content Loss: 4.757929

run [200]:
Style Loss : 4.042702 Content Loss: 4.318666

run [250]:
Style Loss : 3.250791 Content Loss: 4.145459

run [300]:
Style Loss : 2.546430 Content Loss: 4.071249

run [350]:
Style Loss : 1.981914 Content Loss: 4.024672

run [400]:
Style Loss : 1.569494 Content Loss: 3.972436

run [450]:
Style Loss : 1.321036 Content Loss: 3.904739

run [500]:
Style Loss : 1.163311 Content Loss: 3.832359

run [550]:
Style Loss : 1.070525 Content Loss: 3.767200

run [600]:
Style Loss : 1.007565 Content Loss: 3.712166

run [650]:
Style Loss : 0.966649 Content Loss: 3.664161

run [700]:
Style Loss : 0.937533 Content Loss: 3.623591

Building the style transfer model..
Optimizing..
run [50]:
Style Loss : 64.147926 Content Loss: 9.888753

run [100]:
Style Loss : 26.318649 Content Loss: 10.334475

run [150]:
Style Loss : 

KeyboardInterrupt: 